IMPORTANT! Before beginning any lab assignment, be sure to **make your own copy** of the notebook and name it "lastname - Lab 6" or something similar.

# Lab 6: k-Nearest Neighbors (kNN) for Classification and Regression

In this lab, you will work your way through four activities, two of which directly draw on material covered in lecture, and two of which are new. By the end of this lab, you should be able to:

1. Review how to apply kNN for classification tasks using `scikit-learn`
2. (**NEW**) Understand and address the curse of dimensionality
3. (**NEW**) Perform feature selection and hyperparameter tuning to improve model performance
4. Review how to apply kNN for regression tasks

**INSTRUCTIONS**: Read each cell.  Wherever there is a **Question**, answer it in the markdown/text cell below it.  Where-ever there is a **TODO**, complete the code in the code indicated.

## Part A: Simple kNN Classification with scikit-learn

We will start by loading the Wisconsin Breast Cancer dataset, which is a commonly used dataset for classification tasks. This dataset is built-in to `scikit-learn` (meaning no download is required).

The Breast Cancer Wisconsin dataset contains measurements from digitized images of breast mass samples. Each sample has 30 features computed from the cell nuclei, and the task is to classify tumors as malignant (0) or benign (1). You can read more about this dataset [here](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic).

In [ ]:
# Import necessary libraries
import numpy as np
import polars as pl
import pandas as pd
from sklearn.datasets import load_breast_cancer

# Set polars to display all columns and rows when printing a DataFrame
pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_cols(-1)

# Set random seed for reproducibility
np.random.seed(42)

# Load the Breast Cancer dataset built into scikit-learn.  The data
# is returned via .data attribute, although it doesn't know the feature
# names, which are in the .feature_names attribute.  The target feature
# is stored separately and returned via .target attribute.  The mapping of
# its values into names is given by .target_names attribute.
cancer_data = load_breast_cancer()

# Convert into a Polars DataFrame
cancer_df = pl.DataFrame(cancer_data.data)
# When initially loaded, the columns are named "column_0", "column_1", ...,
# "column_29". We can rename them to their original feature names using
# the following code:
column_map = {f"column_{i}": name for i, name in enumerate(cancer_data.feature_names)}
cancer_df = cancer_df.rename(column_map)

# Add the target variable to the DataFrame
cancer_df = cancer_df.with_columns(pl.Series("target", cancer_data.target))

print("Breast Cancer Dataset:")
print(f"Shape: {cancer_df.shape}")
print(f"Features: {cancer_df.columns}")
print("Class distribution:")

# Use Polars expression to perform the counting of different target classes
class_counts = cancer_df.select([
    (pl.col("target") == 0).sum().alias("malignant_count"),
    (pl.col("target") == 1).sum().alias("benign_count")
])

print(f"   Malignant={class_counts.get_column('malignant_count')[0]},")
print(f"      Benign={class_counts.get_column('benign_count')[0]}")

cancer_df.head(10)

Let's build the input features (`X`) and target features (`y`) for this problem and do a little exploratory data analysis.

In [ ]:
# Extract features (X) and target (y)
X_cancer = cancer_df.select(pl.exclude("target")).to_pandas()
y_cancer = cancer_df.select("target").to_numpy().flatten()

# Display first few rows and the rows (tweak pandas display options to 
# show all columns)
pd.set_option("display.max_columns", None)  # Show all columns
print("First 5 samples:")
display(X_cancer.head())

# Check for missing values
print(f"\nMissing values: {X_cancer.isnull().sum().sum()}")

**Question 1:** Compute the feature statistics below, what do you notice about the scales of different features? Why might this be a problem for kNN?

In [ ]:
# TODO: Dump out the Basic statistics on thie dataframe (pandas and polars
# use the same method name here...)

```
ENTER YOUR ANSWER HERE!!!
```

Let's examine the feature distributions for 4 input features for benign and malignant tumors.

In [ ]:
import matplotlib.pyplot as plt

# Visualize feature distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
features_to_plot = ['mean radius', 'mean texture', 'mean area', 'mean smoothness']

for idx, feature in enumerate(features_to_plot):
    ax = axes[idx // 2, idx % 2]
    ax.hist(X_cancer[feature][y_cancer == 0], alpha=0.5, label='Malignant', bins=30)
    ax.hist(X_cancer[feature][y_cancer == 1], alpha=0.5, label='Benign', bins=30)
    ax.set_xlabel(feature)
    ax.set_ylabel('Frequency')
    ax.legend()
    ax.set_title(f'Distribution of {feature}')

plt.tight_layout()
plt.show()

We will split the data into training and testing data, as we did last week with decision trees.  This is a common approach to allow use to test the model on data it has not been trained on.  We will use a 70/30 split and set `random_state=42` for reproducibility.  We are **stratifying** the split to ensure that the class distribution is maintained in both the training and testing sets.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data with a 70/30 split, using random_state=42 for reproducibility 
# and stratify to maintain class distribution
X_train_cancer, X_test_cancer, y_train_cancer, y_test_cancer = train_test_split(
    X_cancer, y_cancer, test_size=0.3, random_state=42, stratify=y_cancer
)

print(f"Training set size: {X_train_cancer.shape[0]}")
print(f"Test set size: {X_test_cancer.shape[0]}")
print(f"Number of features: {X_train_cancer.shape[1]}")


## Baseline kNN Models

When we start a project, we often try to set up a baseline model to "beat".
It basically represents the minimum performance we should expect, with no real effort.
For example, if you were trying to predict a coin flip, a baseline model could be just to always pick "heads".  Assuming a fair coin, 50% of the time you will be correct.  If your model can beat that baseline, it is an improvement.

### Non-Machine Learning Baseline
Given there are a total of 212 Maligant cases in 569 instances, just simply guessing its always cancer (which would be likely to be traumatic to your patients and highly unethical) would lead to a success rate with this dataset of 37.25% (212/569).  You could always assume it is benign and have a success rate of 62.75%, but of course, no one would get the necessary cancer treatment, also unethical. 

Let's see if a machine learning model can improve on guessing...

### Initial kNN Model (Unscaled)

Our initial baseline kNN model will be to simply NOT scale any of the data and use all the features without consideration of their importance.  We will train the model using the training data and assess its accuracy with the testing data.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Set the number of nearest neighbors
k = 3

# TODO: Train kNN on the unscaled X_train_cancer and y_train_cancer data
knn_unscaled = ...
knn_unscaled.fit(...)

# TODO: Compute y_pred_unscaled, which is the predicted classes for X_test_cancer)
y_pred_unscaled = knn_unscaled.predict(...

print("kNN Performance WITHOUT Scaling:")
print(f"Accuracy: {accuracy_score(y_test_cancer, y_pred_unscaled)*100:.2f}%")

Clearly, the kNN model here is doing better than just assuming everything is or is not malignant!

Below we quickly review the standard deviations of the features, which we will use to motivate the need for scaling in the next section.

In [ ]:
# Print the features with the maximum and minimum standard deviation 
# for each feature to get a sense of the scale of the features
std_devs = X_cancer.std()
max_std_feature = std_devs.idxmax()
min_std_feature = std_devs.idxmin()
print(f"\nFeature with maximum standard deviation: {max_std_feature} (std={std_devs[max_std_feature]:.4f})")
print(f"Feature with minimum standard deviation: {min_std_feature} (std={std_devs[min_std_feature]:.4f})")

**Question 2:** What is going to happen to the performance of kNN if both `worst area` and `fractal dimension error` are included as features?  Why?

```
ENTER YOUR ANSWER HERE!!!
```

### Initial kNN Model (Scaled)

Recall we described how to scale/normalize data in lecture.  We will apply the same approach here to see if it improves our kNN model performance.  Since we don't know if there are outliers or not, we will use the `StandardScaler` to scale the data.  This will ensure that all features have a mean of 0 and a standard deviation of 1, putting them on the same scale and allowing kNN to consider all features more equally in its distance calculations.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Define the StandardScaler
scaler = StandardScaler()

# TODO: Fit the scaler on the training data and transform the training data
X_train_scaled = ...

# We only fit the scaler on the training data to avoid data leakage, and
# then we use the same scaler to transform the test data
X_test_scaled = scaler.transform(X_test_cancer)

# Set the number of nearest neighbors
k = 3

# Train kNN WITH scaling
knn_scaled = KNeighborsClassifier(n_neighbors=k)
knn_scaled.fit(X_train_scaled, y_train_cancer)

# TODO: Compute y_pred_scaled, which is the predicted classes for X_test_scaled)
y_pred_scaled = ...

print("kNN Performance WITH Scaling:")
print(f"Accuracy: {accuracy_score(y_test_cancer, y_pred_scaled)*100:.2f}%  ", end="")
print(f"Improvement: {(accuracy_score(y_test_cancer, y_pred_scaled) 
      - accuracy_score(y_test_cancer, y_pred_unscaled))*100:.2f}%")

# Part B: The Curse of Dimensionality

The "curse of dimensionality" refers to the phenomenon where the performance of machine learning models, particularly those based on distance metrics like kNN, degrades as the number of features (dimensions) increases. This happens because as the number of dimensions increases, the data becomes sparse in that many-dimensional space.

For kNN specifically:

- As dimensions increase, distances between points become less meaningful.
- All points tend to become equally far from each other.
- More data is needed to maintain the same density of samples.

Let's demonstrate the curse of dimensionality by testing the quality of a kNN model as we add more features to the dataset.  We will start with just 2 features and then add more features one at a time, observing how the accuracy changes.

### Introducing N-fold Cross-Validation
We will assess performance using N-fold cross-validation on the data, which will give us a more robust estimate of model performance as we change the number of features.  What it does is it splits the ENTIRE data into N parts, trains the model on N-1 of those parts and tests on the remaining part, and repeats this process N times so that each part gets to be the test set once.  The average performance across these N iterations is then reported as the cross-validated performance of the model. So notice this is different than the train/test split we have been doing, which only gives us one estimate of performance.  Cross-validation gives us N estimates of performance and then averages them to get a more robust estimate.

In [ ]:
from sklearn.model_selection import cross_val_score

# Demonstrate curse of dimensionality by testing with different numbers of features
feature_counts = [2, 5, 7, 10, 12, 15, 17, 20, 22, 25, 28, 30]
accuracies = []
cv_scores = []

# Define the number of neighbors for kNN
k = 3

# Define the number of folds for cross-validation
cv_folds = 5

for n_features in feature_counts:
    # Select n random features to avoid biasing towards the first few features,
    # which may be more informative
    np.random.seed(42)  # For reproducibility
    selected_features = np.random.choice(X_train_scaled.shape[1],
                                         n_features, replace=False)
    X_train_subset = X_train_scaled[:, selected_features]
    X_test_subset = X_test_scaled[:, selected_features]

    # Train the model
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_subset, y_train_cancer)
    # Test the model and compute accuracy
    y_pred = knn.predict(X_test_subset)
    accuracies.append(accuracy_score(y_test_cancer, y_pred))

    # Subset the entire dataset for cross-validation as well
    X_subset = X_cancer.iloc[:, selected_features]
    y_subset = y_cancer
    # Also perform an N-fold cross-validation to hopefully get a better
    # sense of the accuracy over N different models
    cv_score = cross_val_score(knn, X_subset, y_subset,
                               cv=cv_folds, scoring='accuracy')
    cv_scores.append(cv_score.mean())

# Plot results
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(feature_counts, accuracies, marker='o', linewidth=2)
plt.xlabel('Number of Features')
plt.ylabel('Test Accuracy')
plt.title('Test Accuracy vs. Number of Features')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(feature_counts, cv_scores, marker='s', color='orange', linewidth=2)
plt.xlabel('Number of Features')
plt.ylabel('Cross-Validation Accuracy')
plt.title('CV Accuracy vs. Number of Features')
plt.grid(True)

plt.tight_layout()
plt.show()

print("Performance with different feature counts:")
for i, n in enumerate(feature_counts):
    print(f"{n:2} features: Test Accuracy = {accuracies[i]*100:.2f}%, "
                            f"CV Accuracy = {cv_scores[i]*100:.2f}%")

**Question 3:** What do you observe in the plots above? Does adding more features always improve performance? Why or why not?

```
ENTER YOUR ANSWER HERE!!!
```

# Part C: Feature Selection to Break the Curse of Dimensionality

We have seen that adding more features can lead to worse performance due to the curse of dimensionality.  However, we were just randomly adding features without considering their relevance to the target variable.  

One way to address the curse of dimensionality is to perform **feature selection**, which involves selecting a subset of the most relevant features for use in model training.  This can help improve model performance by reducing noise and focusing on the most informative features


## How to Automatically Select Features

We want to select the input features that have the strongest relationship with the target variable.  If this was a regression problem (where we predict a continuous value) we could try to use something like a correlation coefficient to measure the strength of the relationship between each feature and the target variable.  However, since this is a classification problem (where we predict a categorical value), we need to use a different approach.

### ANOVA F Test for Feature Selection
There is a test called an **ANOVA F test** that essentially computes "are the mean values of the feature different for the different classes?"  Essentially:
$$
F = \frac{\text{between-class variance}}{\text{within-class variance}}
$$

If the mean values of the input feature are very different for the different target classes, then that feature is likely to be informative for classification.  The ANOVA F test gives us a statistic that measures the strength of the relationship between each feature and the target variable, and we can use this statistic to select the top K features for our kNN model.

### `SelectKBest` for Feature Selection

There is a `SelectKBest` class in `scikit-learn` that can apply a given function to all the features and attempt to determine which features are the most relevant. By selecting the top features based on ANOVA F statistics (the function is `f_classif`), we can potentially improve the performance of our kNN model by reducing the dimensionality of the data and focusing on the most relevant features.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

# Select the top N features based on their ANOVA F-statistic scores 
# with the target variable
topN = 15
selector = SelectKBest(score_func=f_classif, k=topN)

# Fitting the selector to the scaled training data and target labels
# just selects the top N features based on their scores and keeps
# only those features in the transformed data.
selector.fit(X_train_scaled, y_train_cancer)

# Get feature scores into Polars DataFrame for better visualization
feature_scores = pl.DataFrame({
    'Feature': X_cancer.columns,
    'Score': selector.scores_
}).sort('Score', descending=True)

# Plot top N features
plt.figure(figsize=(12, 6))
plt.barh(range(topN), feature_scores['Score'].head(topN))
plt.yticks(range(topN), feature_scores['Feature'].head(topN))
plt.xlabel('ANOVA F-statistic Score')
plt.title(f'Top {topN} Most Important Features')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print(f"\nTop {topN} Most Important Features:")
print(feature_scores.head(topN))

**Question 4**: Go through the code above and make sure you understand how it works.  Specifically, what is the `SelectKBest` class doing and how does it use it to select features?  If you need the documentation for `SelectKBest`, you can find it [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html).

```
ENTER YOUR ANSWER HERE!!!
```

**Question 5**: Now that you know a way to select the most relevant features, how might you decide how many features to select?  What are the trade-offs involved in selecting more or fewer features?

```
ENTER YOUR ANSWER HERE!!!
```

Now examine the following code and see how the performance of the kNN model changes as we add more features based on their importance (as determined by the ANOVA F test).  Do you see an improvement in performance compared to just adding features randomly?

In [ ]:
# Set number of neighbors to use
k = 3
# Set number of folds to use for cross-validation
Nfolds = 5
# Set number of feature to try
N_values = range(1, X_train_cancer.shape[1]+1)

accuracies = []
cv_scores = []

# Find optimal number of features
for topN in N_values:
    # TODO: Select top k (Juan uses "topN") features
    selector = ...

    # Fit and transform the training data
    X_train_selected = selector.fit_transform(X_train_scaled, y_train_cancer)
    # Transform the testing data to select those features
    X_test_selected = selector.transform(X_test_scaled)

    # Train
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_selected, y_train_cancer)

    # Predict values for testing data and evaluate
    y_pred = knn.predict(X_test_selected)
    accuracies.append(accuracy_score(y_test_cancer, y_pred))

    # Subset the entire dataset for cross-validation as well
    X_subset = selector.transform(X_cancer.to_numpy())
    y_subset = y_cancer
    # Cross-validation using N folds to get a better estimate of the 
    # accuracy over N different models using the entire dataset
    cv_score = cross_val_score(knn, X_subset, y_subset,
                               cv=Nfolds, scoring='accuracy')
    cv_scores.append(cv_score.mean())

# Plot
plt.figure(figsize=(12, 5))
plt.plot(N_values, accuracies, marker='o', label='Test Accuracy', linewidth=2)
plt.plot(N_values, cv_scores, marker='s', label='CV Accuracy', linewidth=2)
plt.xlabel('Number of Selected Features (k)')
plt.ylabel('Accuracy')
plt.title('Model Performance vs. Number of Selected Features')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Find optimal k (or Juan calls it optimal N)
optimal_N = N_values[np.argmax(accuracies)]
print(f"\nOptimal number of features: {optimal_N}")
print(f"Best Test Accuracy: {max(accuracies)*100:.2f}%")
print(f"Accuracy with all 30 features: {accuracies[-1]*100:.2f}%")

## Identifying the Final Model with Optimal Features and Optimal k

We have now computed the optimal number of features to use (`optimal_N`).  This is a form of **hyperparameter tuning**.  **Hyperparameters** are parameters that are not learned from the data but are set before the training process.  In this case, the hyperparameter we are tuning is the "number of features to use".  By selecting the optimal number of features, we can improve the performance of our kNN model by reducing the dimensionality of the data and focusing on the most relevant features.

We first build a final subset of training and testing data containing only the optimal features.

In [ ]:
# TODO: Use SelectKbest to select the optimal features (optimal_N) 
selector_final = ...
X_train_final = ...
X_test_final = ...

# Print the selected feature names
selected_features = X_cancer.columns[selector_final.get_support()].tolist()
print(f"Selected {optimal_N} features:")
for i, feat in enumerate(selected_features, 1):
    print(f"{i}. {feat}")

Now let's "tune" the number of neighbors and see how it affects the performance of our final model with the optimal features.  We will plot the performance for different values of $k$ to see if we can find an even better value for $k$ that gives us the best performance on the test set.  We will save this optimal value of $k$ as `optimal_k_neighbors`.

In [ ]:
# Optimize k (number of neighbors)
k_neighbors = range(1, 50, 2) # Odd values from 1 to 49 
train_accuracies = []
test_accuracies = []

for k in k_neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_final, y_train_cancer)
    
    train_accuracies.append(knn.score(X_train_final, y_train_cancer))
    test_accuracies.append(knn.score(X_test_final, y_test_cancer))

plt.figure(figsize=(10, 6))
plt.plot(k_neighbors, train_accuracies, marker='o', label='Training Accuracy', linewidth=2)
plt.plot(k_neighbors, test_accuracies, marker='s', label='Test Accuracy', linewidth=2)
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Accuracy')
plt.title('Model Performance vs. k Value')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

optimal_k_neighbors = k_neighbors[np.argmax(test_accuracies)]
print(f"\nOptimal k (neighbors): {optimal_k_neighbors}")
print(f"Best test accuracy: {max(test_accuracies):.4f}")

**Question 6**: What do you notice about training accuracy vs. test accuracy as $k$ increases? What does this tell you about overfitting?

```
ENTER YOUR ANSWER HERE!!!
```

## Evaluating the Final Model

Finally, we will build our final kNN model using the optimal number of features and the optimal value of $k$, and evaluate its performance on the test set.

In [ ]:
# Train final optimized model
knn_final = KNeighborsClassifier(n_neighbors=optimal_k_neighbors)
knn_final.fit(X_train_final, y_train_cancer)
y_pred_final = knn_final.predict(X_test_final)

print("Final Model Performance:")
print(f"  Accuracy: {accuracy_score(y_test_cancer, y_pred_final)*100:.2f}%")

Last week we also introduced the concept of a **confusion matrix**, which is a table that summarizes the performance of a classification model by showing the counts of true positives, true negatives, false positives, and false negatives.  Let's compute that here

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Confusion Matrix
cm = confusion_matrix(y_test_cancer, y_pred_final)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', 
            xticklabels=cancer_data.target_names,
            yticklabels=cancer_data.target_names)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix - Final kNN Classifier')
plt.tight_layout()
plt.show()

**Question 7:** In a medical context, which is worse: a false positive (predicting malignant when it's benign) or a false negative (predicting benign when it's malignant)? Why?

```
ENTER YOUR ANSWER HERE!!!
```

Once we have computed the confusion matrix, there are other statistics we can use for evaluating classification beyond accuracy, such as:

- **Precision**: The proportion of true positive predictions among all positive predictions. $$Precision = \frac{TP}{TP + FP}$$ It measures how many of the predicted positive cases are actually positive.
- **Recall**: The proportion of true positive predictions among all actual positive cases. $$Recall = \frac{TP}{TP + FN}$$ It measures how many of the actual positive cases were correctly identified by the model.
- **F1 Score**: The harmonic mean of precision and recall. $$F1 = 2 \times \frac{Precision \times Recall}{Precision + Recall}$$ It provides a single metric that balances both precision and recall, especially useful when there is an imbalance in the classes.

We will compute these metrics for our final model to get a more comprehensive understanding of its performance.

**Question 7**: You have 100 patients, 2 of whom has a rare disease.  You build a model that labels 97 out of those hundred people having no disease, correctly labelling the two having the disease, but also one as having the disease who does not.  What is the accuracy of this model?  What is the precision and recall of this model?  What are these telling you about the performance of the model?

```
ENTER YOUR ANSWER HERE!!!
```

Now let's compute these statistics for our final kNN model on the test set to see how well it performs in terms of precision, recall, and F1 score.

In [ ]:
from sklearn.metrics import classification_report

print("\nClassification Report:")
print(classification_report(y_test_cancer, y_pred_final, target_names=cancer_data.target_names))

**Question 8**: This model has higher precision than recall.  What does this mean in terms of the types of errors the model is making?  As a doctor trying to detect cancer, would you prefer a model with higher precision or higher recall?

```
ENTER YOUR ANSWER HERE!!!
```

## Part D: kNN for Regression

Just as kNN can be used for classification tasks, it can also be applied to regression tasks, where the goal is to predict a continuous value rather than a categorical class.  In kNN regression, the predicted value for a new data point is typically computed as the average (or sometimes a weighted average) of the target values of its k nearest neighbors in the feature space.

We will explore the Diabetes dataset, which is a commonly used dataset for regression tasks. This dataset contains 10 features (age, sex, BMI, blood pressure, and 6 blood serum measurements) for 442 diabetes patients. The target is a quantitative measure of disease progression one year after baseline. You can read more about this dataset [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html#sklearn.datasets.load_diabetes) and [here](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html).

In [ ]:
from sklearn.datasets import load_diabetes

# Load the Diabetes dataset, which is a regression dataset
diabetes_data = load_diabetes()

# Convert into a Polars DataFrame
diabetes_df = pl.DataFrame(diabetes_data.data)
# When initially loaded, the columns are named "column_0", "column_1", ...,
# "column_29". We can rename them to their original feature names using
# the following code:
column_map = {f"column_{i}": name for i, name in enumerate(diabetes_data.feature_names)}
diabetes_df = diabetes_df.rename(column_map)

# Add the target variable to the DataFrame
diabetes_df = diabetes_df.with_columns(pl.Series("target", diabetes_data.target))

print("Diabetes Dataset:")
print(f"Shape: {diabetes_df.shape}")
print(f"Features: {diabetes_df.columns}")

diabetes_df.head(10)

Now you should explore the statistics for the Diabetes dataset and be sure you understand if the features are on different scales and whether scaling is necessary for kNN regression. 

In [ ]:
# TODO Select the features (X) and target (y) from the diabetes dataset
X_diabetes = ...
y_diabetes = ...

# TODO: Check if there are missing values in the diabetes dataset


# TODO: Print the basic statistics for the diabetes dataset to understand the 
# scale of the features and whether scaling is necessary for kNN regression.


**Question 9**: Do you think scaling is necessary for kNN regression on the Diabetes dataset? Why or why not?

```
ENTER YOUR ANSWER HERE!!!
```

**Question 10:** If you examine the statistics computed in the cell below for the target variable (disease progression), you can see that it is clearly NOT standardized and is on a different scale than the input features.  Is this a problem for kNN regression? Why or why not?

In [ ]:
import matplotlib.pyplot as plt

print(f"\nTarget statistics:")
print(f"  Mean: {y_diabetes.mean():.2f}")
print(f"  Std: {y_diabetes.std():.2f}")
print(f"  Min: {y_diabetes.min():.2f}")
print(f"  Max: {y_diabetes.max():.2f}")

# Target distribution
plt.hist(y_diabetes, bins=30, edgecolor='black')
plt.xlabel('Disease Progression')
plt.ylabel('Frequency')
plt.title('Distribution of Target Variable')
plt.show()

```
ENTER YOUR ANSWER HERE!!!
```

Let's split up the data into training and testing data and then build a kNN regression model to predict disease progression based on the input features. 

In [ ]:
# TODO Split the data (use a 70/30 split and random_state=42 for reproducibility,
# as this is a regression dataset, we don't need to stratify)
X_train_diabetes, X_test_diabetes, y_train_diabetes, y_test_diabetes = ...

print(f"Training set size: {X_train_diabetes.shape[0]}")
print(f"Test set size: {X_test_diabetes.shape[0]}")
print(f"Number of features: {X_train_diabetes.shape[1]}")

Because this is a regression problem where the target variable is continuous, we will evaluate the performance of our kNN regression model using different metrics than classification (after all, what would "accuracy" mean here?).

Instead, we will use metrics such as

- **Mean Absolute Error (MAE)**: The average of the absolute differences between the predicted values and the actual values. It gives an idea of how much the predictions deviate from the actual values on average.
- **Root Mean Squared Error (RMSE)**: The square root of the average of the squared differences between the predicted values and the actual values. It penalizes larger errors more than smaller ones, making it sensitive to outliers.
- **R-squared ($R^2$)**: A statistical measure that represents the proportion of the variance in the target variable that is explained by the input features. It ranges from 0 to 1, with higher values indicating better model performance.  In essence, it is the correlation coefficient squared between the predicted values and the actual values.

If we apply these to compare the actual values of disease progression to the predicted values from our kNN regression model, we can get a sense of how well our model is performing in terms of predicting the continuous target variable.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Set number of neighbors to use for kNN regression
k = 5

# Train baseline model
knn_regressor = KNeighborsRegressor(n_neighbors=k)
knn_regressor.fit(X_train_diabetes, y_train_diabetes)
y_pred_diabetes = knn_regressor.predict(X_test_diabetes)

# Evaluate
mse = mean_squared_error(y_test_diabetes, y_pred_diabetes)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_diabetes, y_pred_diabetes)
r2 = r2_score(y_test_diabetes, y_pred_diabetes)

print("Baseline kNN Regression Performance (k=5):")
print(f"  RMSE: {rmse:.2f}")
print(f"  MAE: {mae:.2f}")
print(f"  R² Score: {r2:.4f}")
print(f"\n(Target std: {y_diabetes.std():.2f})")

In [ ]:
# Visualize predictions
plt.figure(figsize=(12, 5))

# Predicted vs Actual
plt.subplot(1, 2, 1)
plt.scatter(y_test_diabetes, y_pred_diabetes, alpha=0.6, edgecolors='k')
plt.plot([y_test_diabetes.min(), y_test_diabetes.max()], 
         [y_test_diabetes.min(), y_test_diabetes.max()], 
         'r--', lw=2, label='Perfect Prediction')
plt.fill_between([y_test_diabetes.min(), y_test_diabetes.max()],
                 [y_test_diabetes.min() - rmse, y_test_diabetes.max() - rmse],
                 [y_test_diabetes.min() + rmse, y_test_diabetes.max() + rmse],
                 alpha=0.2, color='red', label=f'RMSE ±{rmse:.0f}')
plt.xlabel('Actual Disease Progression')
plt.ylabel('Predicted Disease Progression')
plt.title('Predicted vs Actual Values')
plt.legend()
plt.grid(True, alpha=0.3)

# Residuals
plt.subplot(1, 2, 2)
residuals = y_test_diabetes - y_pred_diabetes
plt.scatter(y_pred_diabetes, residuals, alpha=0.6, edgecolors='k')
plt.axhline(y=0, color='r', linestyle='--', lw=2)
plt.xlabel('Predicted Disease Progression')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Examine the plot above and remember the meanings of the metrics we just discussed.

- **Mean Absolute Error (MAE)**: In essence, the average of how far off the dashed red line the points are on average (above being positive errors and below being negative errors).
- **Root Mean Squared Error (RMSE)**: In essence, it is the average of how far off the dashed red line the points are (where the direction of the error does not matter).
- **R-squared ($R^2$)**: In essence, the correlation coefficient squared between the predicted values and the actual values.

What do these metrics tell you about the performance of the kNN regression model on the Diabetes dataset?  

- The MAE of 44 tells me the model tends to overpredict the target variable by about 44 units on average. 
- The RMSE of 57 on the scale of 300 suggests a typical error well over 10%. 
- The R-squared value of 0.4 indicates that the model explains about 40% of the variance in the target variable, which suggests that there is still a significant amount of variance that is not captured by the model. 

Overall, these metrics suggest that while the kNN regression model has some predictive power, there is room for improvement in terms of accuracy and explanatory power.

## Tuning the kNN Regression Model Hyperparameters

Given the smallish number of features in the Diabetes dataset, we will not worry about feature selection here, but we will still want to tune the hyperparameters of the kNN regression model to see if we can improve performance. For now we'll focus on the number of neighbors ($k$).

In the cell below, try setting up the loop to train and evaluate kNN regression models with different values of $k$ (e.g., from 1 to 30) and save the performance metrics of RMSE and R-squared as a function of $k$.  This will allow you to visually inspect how the choice of $k$ affects the performance of the model and help you identify the optimal value of $k$ for this regression task.

We will also perform N-fold cross-validation to get a more robust estimate of model performance for each value of $k$ and save those cross-validated R-squared scores as well.

In [ ]:
# Test different k values
k_values_reg = range(1, 51,2)

# Number of folds for cross-validation
Nfolds = 5

# Save the RMSE and R^2 scores for each k value to analyze the performance
train_rmse = []
test_rmse = []
test_r2 = []
cv_rmse = []

for k in k_values_reg:
    # Train kNN regressor with current k
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train_diabetes, y_train_diabetes)
    
    # Compute the Training RMSE and save the RMSE for the training data
    y_train_pred = knn.predict(X_train_diabetes)
    train_rmse.append(np.sqrt(mean_squared_error(y_train_diabetes, y_train_pred)))
    
    # Compute and save the test data RMSE
    y_test_pred = knn.predict(X_test_diabetes)
    test_rmse.append(np.sqrt(mean_squared_error(y_test_diabetes, y_test_pred)))

    # Compute and save the R^2 score for the test data
    test_r2.append(r2_score(y_test_diabetes, y_test_pred))

    # Cross-validation of the negative RMSE (scores are normally maximized,
    # we will take the negative of this to get the actual RMSE).  Remember
    # we use the entire dataset for cross-validation, not just the training 
    # data, to get a better estimate of the model's performance across 
    # different subsets of the data.
    cv_score = -1*cross_val_score(knn, X_diabetes, y_diabetes,
                               cv=Nfolds, scoring='neg_root_mean_squared_error')
    cv_rmse.append(cv_score.mean())

In [ ]:
# Plot results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# RMSE plot
axes[0].plot(k_values_reg, train_rmse, marker='o', label='Training RMSE', linewidth=2)
axes[0].plot(k_values_reg, test_rmse, marker='s', label='Test RMSE', linewidth=2)
axes[0].plot(k_values_reg, cv_rmse, marker='s', color='blue', label='CV RMSE', linewidth=2)
axes[0].set_xlabel('Number of Neighbors (k)')
axes[0].set_ylabel('RMSE')
axes[0].set_title('RMSE vs. k Value')
axes[0].legend()
axes[0].grid(True)

# R^2 plot
axes[1].plot(k_values_reg, test_r2, marker='o', color='green', linewidth=2, label='Test R^2')
axes[1].set_xlabel('Number of Neighbors (k)')
axes[1].set_ylabel('R^2 Score')
axes[1].set_title('R^2 Score vs. k Value')
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Find optimal k based on the minimum test RMSE
optimal_k_reg = k_values_reg[np.argmin(test_rmse)]
print(f"\nOptimal k: {optimal_k_reg}")
print(f"Best test RMSE: {min(test_rmse):.2f}")
print(f"Best test R^2: {test_r2[np.argmin(test_rmse)]:.4f}")

Let's now fit the final optimized model with the optimal number of neighbors and evaluate its performance on the test set.  We will compute the MAE, RMSE, and R-squared for this final model to see how well it performs in predicting disease progression.

In [ ]:
# Train final model
knn_final_reg = KNeighborsRegressor(n_neighbors=optimal_k_reg)
knn_final_reg.fit(X_train_diabetes, y_train_diabetes)
y_pred_final_reg = knn_final_reg.predict(X_test_diabetes)

# Final evaluation
final_rmse = np.sqrt(mean_squared_error(y_test_diabetes, y_pred_final_reg))
final_mae = mean_absolute_error(y_test_diabetes, y_pred_final_reg)
final_r2 = r2_score(y_test_diabetes, y_pred_final_reg)

print(f"Final Optimized kNN Regression Model (k={optimal_k_reg}):")
print(f"  RMSE: {final_rmse:.2f}")
print(f"  MAE: {final_mae:.2f}")
print(f"  R^2 Score: {final_r2:.4f}")

# Cross-validation
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(knn_final_reg, X_train_diabetes, y_train_diabetes, 
                            cv=5, scoring='neg_root_mean_squared_error')
print(f"\nCross-validation RMSE: {-cv_scores.mean():.2f} (+/- {cv_scores.std():.2f})")

In [ ]:
# Final visualization
plt.figure(figsize=(12, 5))

# Predicted vs Actual with error bands
plt.subplot(1, 2, 1)
plt.scatter(y_test_diabetes, y_pred_final_reg, alpha=0.6, edgecolors='k', s=50)
plt.plot([y_test_diabetes.min(), y_test_diabetes.max()], 
         [y_test_diabetes.min(), y_test_diabetes.max()], 
         'r--', lw=2, label='Perfect Prediction')
plt.fill_between([y_test_diabetes.min(), y_test_diabetes.max()],
                 [y_test_diabetes.min() - final_rmse, y_test_diabetes.max() - final_rmse],
                 [y_test_diabetes.min() + final_rmse, y_test_diabetes.max() + final_rmse],
                 alpha=0.2, color='red', label=f'RMSE ±{final_rmse:.0f}')
plt.xlabel('Actual Disease Progression')
plt.ylabel('Predicted Disease Progression')
plt.title(f'Final Model: Predicted vs Actual (R^2={final_r2:.3f})')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
residuals = y_test_diabetes - y_pred_final_reg
plt.scatter(y_pred_final_reg, residuals, alpha=0.6, edgecolors='k')
plt.axhline(y=0, color='r', linestyle='--', lw=2)
plt.xlabel('Predicted Disease Progression')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Question 12**: Can you confirm this model performed better than the initial kNN regression model we built at the beginning of this section?  How do you know?

```
ENTER YOUR ANSWER HERE!!!
```

### Remaining Questions to Consider (No need to answer these, just think about them)

1. We used the test RMSE to select the optimal value of $k$ for our final model.  Could the cross-validation RMSE been used instead?  Why or why not?
2. Another hyperparameter we could have tuned is the distance metric used to compute the nearest neighbors (e.g., Euclidean distance, Manhattan distance, etc.).  How might changing the distance metric affect the performance of the kNN regression model?
3. We could also have tried weighting the neighbors differently (e.g., giving more weight to closer neighbors).  How might this affect the performance of the kNN regression model?

## What you need to submit

Once you have completed this lab, you need to submit your work. You should submit the `.ipynb` notebook file. To do this, go to the File menu and select **Download > Download .ipynb** (this is the native Jupyter notebook format).  Submit that file to the Lab 6 dropbox on D2L.